## Codificación One-Hot ( One-Hot encoding OHE)

La codificación One-Hot consiste en codificar cada una de las variables categóricas con una variable booleana o binaria (también llamada variable dummy) la cual toma el valor de 0 o 1, indicando si la categoría esta presente en una observación.

Por ejemplo, para una variable categórica 'género" con las etiquetas 'femenino' y 'masculino', podemos generar la variable binaria 'femenino' que toma los valores de 1 si la persona es una mujer o 0 si no lo es. Alternativamente, podemos generar la variable 'masculino', que toma el valor de 1 si la persona es un hombre y 0 de lo contrario.

Para la variable categórica 'color' con los valores 'rojo', 'azul' y 'verde' podemos crear 3 nuevas variables llamadas: 'rojo', 'azul' y 'verde. Estas variables toman el valor de 1 si la observación en cuestión es de ese color o el valor de 0 de lo contrario.

### Codificación en k-1 variables dummy

Es importante resaltar, que para la variable color, si creamos solo dos variables binarias, por ejemplo  'roja' y 'azul', estamos codificando **TODA** la información:

- si la observación es roja, será capturada por la variable 'roja' ( roja =1 y azul = 0)
- si la observación es azul, será capturada por la variable 'azul' ( roja =0 y azul = 1)
- si la observación es verde, será capturada por la combinación de las variables 'rojo' y 'azul' ( roja =0 y azul = 0)

Por lo tanto no necesitamos añadir la tercera variable 'verde' para capturar que la observación es verde.

Generalmente, una variable categórica se puede codificar creando k-1 variables binarias, donde k es el número de categorías distintas o únicas en la variable. En el caso de género, k=1, (masculino/femenino) por lo tanto solo necesitamos crear una variable binaria (k-1 = 1). En el caso de color, que tiene 3 categorías diferentes (k=3), necesitamos crear 2 variables binarias (k-1 = 2)  para capturar toda la información.

La codificación One-Hot en k-1 variables binarias tiene en cuenta que podemos usar una dimensión menos y todavía representar toda la información: si la observación es 0 en todas las k-1 variables binarias, entonces sería 1 en la k variable binaria (que no está presente y que por lo tanto podemos considerar redundante).

**En la codificación one-hot, creamos k-1 variables binarias por cada variable categórica**


La mayoría de los algoritmos de machine learning, consideran todo el conjunto de datos cuando se esta ajustando sus parámetros, por lo tanto es mejor evitar introducir información redundante, y utilizar solo k-1 variables binarias para codificar las categorías de una variable.


### Excepción: codificación one-hot en k variables dummy

Hay unas pocas ocasiones cuando es mejor codificar las categorías en k variables dummy o binarias.

- cuando se construyen algoritmos basados en árboles
- cuando utilizamos algoritmos recursivos para seleccionar variables
- cuando estamos interesados en determinar la importancia de cada categoría por separado.

Los algoritmos basados en árboles, a diferencia del resto de los algoritmos de machine learning, **no evalúan** el conjunto de datos en su totalidad durante el proceso de entrenamiento. Estos algoritmos aleatoriamente extraen un sub-conjunto de variables de los datos por cada uno de los nodos del árbol. Por lo tanto, si queremos que el algoritmo considere **todas** las categorías, es necesario codificar las variables categóricas en **k variables binarias**.

Si estamos planeando utilizar métodos de selección de variables basados en eliminación (o adición) recursiva, o si queremos evaluar la importancia de cada etiqueta de una variable categórica, necesitamos utilizar todo el conjunto de variables binarias (k) para que los algoritmos de machine learning seleccionen cuales variables tienen el mayor poder predictivo.


### Ventajas de la codificación one-hot

- Fácil de implementar
- No asume/ impone condiciones sobre la distribución de la variable o sus categorías
- Mantiene toda la información de la variable categórica
- Es apropiada para modelos lineales

### Limitaciones

- Expande el espacio de las variables
- No añade información adicional con la codificación
- Muchas variables dummy pueden ser idénticas, introduciendo información redundante en las variables

### Importante

Si nuestros datos tienen algunas variables con alta cardinalidad, fácilmente podemos terminar con un conjunto de datos con miles de columnas, que puede hacer el entrenamiento de algoritmos bastante lento, y la interpretación de los modelos bastante difícil.

Adicionalmente, muchas de las variables dummy pueden ser similares la una con la otra, ya que no es inusual que 2 o más variables compartan la misma combinación de 1s y 0s. Por lo tanto, la codificación one-hot puede introducir información redundante o duplicada asi utilicemos la codificación con k-1 variables binarias.


## En este demo:

Vamos a ver como realizar la codificación one-hot con:
- pandas
- Scikit-learn
- Feature-Engine

y estudiaremos las ventajas y limitaciones de cada una de las implementaciones usando el Titanic dataset.


In [1]:
import pandas as pd

# separar datasets
from sklearn.model_selection import train_test_split

# para codificación one hot con sklearn
from sklearn.preprocessing import OneHotEncoder

# para codificación one hot con feature-engine
from feature_engine.encoding import OneHotEncoder as fe_OHE

In [2]:
# carguemos el titanic dataset

data = pd.read_csv('../titanic.csv',
                   usecols=['sex', 'embarked', 'cabin', 'survived'])
data.head()

,survived,sex,cabin,embarked
0,1,female,B5,S
1,1,male,C22,S
2,0,female,C22,S
3,0,male,C22,S
4,0,female,C22,S


In [3]:
# capturemos solo la primera letra de la variable 'cabin' 
# para esta demostración

data['cabin'] = data['cabin'].str[0]

data.head()

,survived,sex,cabin,embarked
0,1,female,B,S
1,1,male,C,S
2,0,female,C,S
3,0,male,C,S
4,0,female,C,S


### Importante

Como en la sustitución, todos los métodos de codificación de categorías deben ser realizados en el set de entrenamiento y luego propagados al set de prueba.


#### Por qué?

Estos métodos aprenden los patrones del set de entrenamiento, por lo tanto queremos evitar pasar información adicional (data leakage) y sobre-ajustar el modelo. La razón más importante, sin embargo, es porque no sabemos si en los datos futuros, tendremos todas las categorías que estuvieron presentes en el set de entrenamiento o si tendremos mas o menos categorías. Por lo tanto, queremos anticiparnos a esta incertidumbre fijando desde el principio los procesos adecuados. Queremos crear transformadores que aprendan de las categorías del set de entrenamiento y usar ese aprendizaje para crear las variables dummy en ambos sets: entrenamiento y prueba.

**'Data Leakage'** es cuando información fuera del set de entrenamiento es usada para crear el modelo. Sucede cuando por alguna razón el modelo aprende de datos que no deberían estar disponibles en un escenario real, por ejemplo los datos 'futuros' o los datos de cuando el modelo esté en producción.
En otras palabras, cuando los datos usados para entrenar un modelo de machine learning tienen la información que estamos intentando predecir.


In [4]:
# separemos los datos en sets de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['sex', 'embarked', 'cabin']],  # predictores
    data['survived'],  # target
    test_size=0.3,  # porcentage de observaciones en el set de prueba
    random_state=0)  # semilla para asegurar reproducibilidad

X_train.shape, X_test.shape

((916, 3), (393, 3))

### Exploremos cardinalidad

In [5]:
# la variable sex tiene dos etiquetas

X_train['sex'].unique()

array(['female', 'male'], dtype=object)

In [6]:
# embarked tiene 3 etiquetas y datos ausentes

X_train['embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [7]:
# cabin tiene 9 etiquetas y datos ausentes

X_train['cabin'].unique()

array([nan, 'E', 'C', 'D', 'B', 'A', 'F', 'T', 'G'], dtype=object)

## Codificación One hot con pandas

### Ventajas

- rápido
- devuelve un dataframe de pandas 
- devuelve los nombres de las variables para las variables dummy 

### Limitaciones de pandas:

- no preserva la información del set de entrenamiento para propagar al set de prueba.

-----

El método de pandas get_dummies(), crea variables binarias como categorías en la variable.

Si la variable color tiene 3 categorías en el segmento de entrenamiento, crea dos variables dummy. Sin embargo, si la variable color tiene 5 categorías en el set de prueba, crea 4 variables binarias;  por lo tanto los sets de entrenamiento y prueba terminarían con un número diferente de variables lo cual sería incompatible con el entrenamiento de un modelo y 'scoring' usando scikit-learn.

En la práctica, no deberíamos usar el método get_dummies() en nuestro pipeline de machine learning. Sin embargo, es bueno si queremos hacer una exploración de los datos rápida. Veamos esto con unos ejemplos.


###  k variables dummy 

In [8]:
# podemos crear variables dummy usando el método de 
# pandas get_dummies

tmp = pd.get_dummies(X_train['sex'])

tmp.head()

,female,male
501,1,0
588,1,0
402,1,0
1193,0,1
686,1,0


In [9]:
# para facilitar la visualización coloquemos las variables dummies al lado
# de la variable original
pd.concat([X_train['sex'],
           pd.get_dummies(X_train['sex'])], axis=1).head()

,sex,female,male
501,female,1,0
588,female,1,0
402,female,1,0
1193,male,0,1
686,female,1,0


In [10]:
# repitamos para embarked 

tmp = pd.get_dummies(X_train['embarked'])

tmp.head()

,C,Q,S
501,0,0,1
588,0,0,1
402,1,0,0
1193,0,1,0
686,0,1,0


In [11]:
# para facilitar visualización concatenemos variable 
# original y transformada

pd.concat([X_train['embarked'],
           pd.get_dummies(X_train['embarked'])], axis=1).head()

,embarked,C,Q,S
501,S,0,0,1
588,S,0,0,1
402,C,1,0,0
1193,Q,0,1,0
686,Q,0,1,0


In [12]:
# y ahora para cabin

tmp = pd.get_dummies(X_train['cabin'])

tmp.head()

,A,B,C,D,E,F,G,T
501,0,0,0,0,0,0,0,0
588,0,0,0,0,0,0,0,0
402,0,0,0,0,0,0,0,0
1193,0,0,0,0,0,0,0,0
686,0,0,0,0,0,0,0,0


In [13]:
# y ahora para todas las variables juntas: set de entrenamiento 

tmp = pd.get_dummies(X_train)

print(tmp.shape)

tmp.head()

(916, 13)


,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,cabin_A,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_T
501,1,0,0,0,1,0,0,0,0,0,0,0,0
588,1,0,0,0,1,0,0,0,0,0,0,0,0
402,1,0,1,0,0,0,0,0,0,0,0,0,0
1193,0,1,0,1,0,0,0,0,0,0,0,0,0
686,1,0,0,1,0,0,0,0,0,0,0,0,0


In [14]:
# todas las variables en: el set de prueba

tmp = pd.get_dummies(X_test)

print(tmp.shape)

tmp.head()

(393, 12)


,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,cabin_A,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G
1139,0,1,0,0,1,0,0,0,0,0,0,0
533,1,0,0,0,1,0,0,0,0,0,0,0
459,0,1,0,0,1,0,0,0,0,0,0,0
1150,0,1,0,0,1,0,0,0,0,0,0,0
393,0,1,0,0,1,0,0,0,0,0,0,0


La ventaja de usar pandas get_dummies:
- el dataframe resultante contiene el nombre de las variables

**y las limitaciones:**

El set de entrenamiento tiene 13 variables dummy, mientras que el set de prueba tiene 12 variables. Eso sucede porque no había categoría T en la variable 'cabin' en el set de prueba.

Esto podría causar problemas si el entrenamiento y el 'scoring' o evaluación del modelo se hace con scikit-learn, porque los predictores requieren que los sets de entrenamiento y prueba tengan el mismo tamaño.


###  k -1 etiquetas 

In [15]:
# obtener k-1 etiquetas: necesitamos indicar a get_dummies
# eliminar la primera variable binaria

tmp = pd.get_dummies(X_train['sex'], drop_first=True)

tmp.head()

,male
501,0
588,0
402,0
1193,1
686,0


In [16]:
# obtener k-1 etiquetas: necesitamos indicar a get_dummies
# eliminar la primera variable binaria

tmp = pd.get_dummies(X_train['embarked'], drop_first=True)

tmp.head()

,Q,S
501,0,1
588,0,1
402,0,0
1193,1,0
686,1,0


Para embarked, si una observacion muestra 0 para Q y S, entonces su valor debe ser C, para la categoria restante.

Caveat: esta variable tiene datos faltantes, entonces a no ser que codifiquemos los datos faltantes, toda la información contenida en la varaible no se capturará.

In [17]:
# todas las variables: set de entrenamiento

tmp = pd.get_dummies(X_train, drop_first=True)

print(tmp.shape)

tmp.head()

(916, 10)


,sex_male,embarked_Q,embarked_S,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_T
501,0,0,1,0,0,0,0,0,0,0
588,0,0,1,0,0,0,0,0,0,0
402,0,0,0,0,0,0,0,0,0,0
1193,1,1,0,0,0,0,0,0,0,0
686,0,1,0,0,0,0,0,0,0,0


In [18]:
# todas las variables: set de prueba

tmp = pd.get_dummies(X_test, drop_first=True)

print(tmp.shape)

tmp.head()

(393, 9)


,sex_male,embarked_Q,embarked_S,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G
1139,1,0,1,0,0,0,0,0,0
533,0,0,1,0,0,0,0,0,0
459,1,0,1,0,0,0,0,0,0
1150,1,0,1,0,0,0,0,0,0
393,1,0,1,0,0,0,0,0,0


### Bono: get_dummies() puede manejar variables dummy para indicar datos faltantes

In [19]:
# podemos addicionar una variable dummy addicional para indicar datos faltantes

pd.get_dummies(X_train['embarked'], drop_first=True, dummy_na=True).head()

,Q,S,NaN
501,0,1,0
588,0,1,0
402,0,0,0
1193,1,0,0
686,1,0,0


## Codificación one-hot con Scikit-learn

### Ventajas

- rápido
- crea el mismo número de variables en el set de prueba y entrenamiento

### Limitaciones

- devuelve un arregloe numpy ('numpy array') en lugar de un dataframe de pandas 
- no devuelve el nombre de las variables, por lo tanto no es conveniente para la exploración de las variables


In [20]:
# creemos y entrenemos el encoder

encoder = OneHotEncoder(categories='auto',
                       drop='first', # devuelve k-1, usa drop=false para devolver k dummies
                       sparse=False)

encoder.fit(X_train.fillna('Missing'))

OneHotEncoder(drop='first', sparse=False)

In [21]:
# miremos las categorias aprendidas

encoder.categories_

[array(['female', 'male'], dtype=object),
 array(['C', 'Missing', 'Q', 'S'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'Missing', 'T'], dtype=object)]

In [22]:
# transformemos el set de prueba

tmp = encoder.transform(X_test.fillna('Missing'))

pd.DataFrame(tmp).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Si vemos las categorías, podemos derivar que la primera columna es 'male' - masculino y las columnas 1-3 son 'Missing', 'Q' y 'S', respectivamente. Sin embargo, requiere que manualmente agreguemos el nombre a cada de las variables.

Hay dos formas de hacer esto, por ejemplo, crear 1 transformador one-hot por cada variable. Recomendamos este blog para ver un ejemplo:
http://www.insightsbot.com/blog/McTKK/python-one-hot-encoding-with-scikit-learn

Sin embargo, no somos fanáticas de esta implementación, y preferimos usar feature-engine.


In [23]:
# podemos empezar transformando el set de prueba

tmp = encoder.transform(X_test.fillna('Missing'))

pd.DataFrame(tmp).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Podemos ver que los sets de entrenamiento y prueba tiene el mismo numero de variables

Para más detalles acerca del transformador OneHotEncoder de Scikit-learn 
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html


## Codificación One hot con Feature-Engine

### Ventajas

- rápido
- devuelve un dataframe
- devuelve el nombre de las variables
- permite seleccionar las variables a codificar

### Limitaciones
- todavía esta por verse ;)


In [24]:
ohe_enc = fe_OHE(
    top_categories=None,
    variables=['sex', 'embarked'], # podemos seleccionar las variables a codificar
    drop_last=True) # devuelve k-1, usar False para devolver k


ohe_enc.fit(X_train.fillna('Missing'))

OneHotEncoder(drop_last=True, variables=['sex', 'embarked'])

In [25]:
tmp = ohe_enc.transform(X_train.fillna('Missing'))

tmp.head()

,cabin,sex_female,embarked_S,embarked_C,embarked_Q
501,Missing,1,1,0,0
588,Missing,1,1,0,0
402,Missing,1,0,1,0
1193,Missing,0,0,0,1
686,Missing,1,0,0,1


Ahora feature-engine devuelve las variables dummy con sus nombres, y elimina la variable original, dejando el conjunto de datos listo para seguir explorando o constuir modelos de machine learning.

In [26]:
tmp = ohe_enc.transform(X_test)

tmp.head()

,cabin,sex_female,embarked_S,embarked_C,embarked_Q
1139,NaN,0,1,0,0
533,NaN,1,1,0,0
459,NaN,0,1,0,0
1150,NaN,0,1,0,0
393,NaN,0,1,0,0


In [27]:
# Codificador One Hot de Feature-Engine también selecciona
# todas las variables categóricas automáticamente

ohe_enc = fe_OHE(
    top_categories=None,
    drop_last=True) # devuelve k-1, false para devolver k


ohe_enc.fit(X_train.fillna('Missing'))

OneHotEncoder(drop_last=True, variables=['sex', 'embarked', 'cabin'])

In [28]:
ohe_enc.variables

['sex', 'embarked', 'cabin']

In [29]:
tmp = ohe_enc.transform(X_train.fillna('Missing'))

tmp.head()

,sex_female,embarked_S,embarked_C,embarked_Q,cabin_Missing,cabin_E,cabin_C,cabin_D,cabin_B,cabin_A,cabin_F,cabin_T
501,1,1,0,0,1,0,0,0,0,0,0,0
588,1,1,0,0,1,0,0,0,0,0,0,0
402,1,0,1,0,1,0,0,0,0,0,0,0
1193,0,0,0,1,1,0,0,0,0,0,0,0
686,1,0,0,1,1,0,0,0,0,0,0,0


In [30]:
tmp = ohe_enc.transform(X_test.fillna('Missing'))

tmp.head()

,sex_female,embarked_S,embarked_C,embarked_Q,cabin_Missing,cabin_E,cabin_C,cabin_D,cabin_B,cabin_A,cabin_F,cabin_T
1139,0,1,0,0,1,0,0,0,0,0,0,0
533,1,1,0,0,1,0,0,0,0,0,0,0
459,0,1,0,0,1,0,0,0,0,0,0,0
1150,0,1,0,0,1,0,0,0,0,0,0,0
393,0,1,0,0,1,0,0,0,0,0,0,0


Observa como este codificador devuelve una variable cabin_T para el set de prueba también, a pesar que esta categoría no esta presente en el set de prueba. Esto permite la integración con un 'pipeline' de Scikit-learn y la evaluación del modelo en el set de prueba con el algoritmo escogido.
De hecho podemos revisar que la suma de cabin_t es de hecho 0:


In [31]:
tmp['cabin_T'].sum()

0